# <span style="color:#2C786C">2. 프로젝트: SentencePiece 사용하기</span>

## <span style="color:#F7B400">순서</span>
> **<span style="color:#2C786C">Step 1. SentencePiece 설치하기</span>**<br>
**<span style="color:#2C786C">Step 2. SentencePiece 모델 학습</span>**<br>
**<span style="color:#2C786C">Step 3. Tokenizer 함수 작성</span>**<br>
**<span style="color:#2C786C">Step 4. 네이버 영화리뷰 감정분석 문제에 SentencePiece 적용해 보기</span>**<br>
**<span style="color:#2C786C">Step 5. 루브릭 평가</span>**<br>
**<span style="color:#2C786C">Step 6. 회고</span>**

## <span style="color:#124445">Step 1. SentencePiece 설치하기</span>
---
SentencePiece는 Google에서 제공하는 오픈소스 기반 Sentence Tokenizer/Detokenizer 로서, BPE와 unigram 2가지 subword 토크나이징 모델 중 하나를 선택해서 사용할 수 있도록 패키징한 것입니다. 아래 링크의 페이지에서 상세한 내용을 파악할 수 있습니다.

* [google/sentencepiece](https://github.com/google/sentencepiece)

위 페이지의 서두에서도 언급하고 있듯, SentencePiece는 딥러닝 자연어처리 모델의 앞부분에 사용할 목적으로 최적화되어 있는데, 최근 pretrained model들이 대부분 SentencePiece를 tokenizer로 채용하면서 사실상 표준의 역할을 하고 있습니다. 앞으로의 실습 과정에서 자주 만나게 될 것이므로 꼭 친숙해지시기를 당부드립니다.

다음과 같이 설치를 진행합니다. SentencePiece는 python에서 쓰라고 만들어진 라이브러리는 아니지만 편리한 파이썬 wrapper를 아래와 같이 제공하고 있습니다.

    $ pip install sentencepiece

## <span style="color:#124445">Step 2. SentencePiece 모델 학습</span>
---
앞서 배운 **`tokenize()`** 함수를 기억하나요? 다시 한번 상기시켜드릴게요!

```{.python}
def tokenize(corpus):  # corpus: Tokenized Sentence's List
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer
```

위와 같이 **`tf.keras.preprocessing.text.Tokenizer`**에 corpus을 주고 **`tokenizer.fit_on_texts(corpus)`**을 하면 토크나이저 내부적으로 단어 사전과 토크나이저 기능을 corpus에 맞춤형으로 자동생성해 주는 것입니다.

그럼 이를 위해서 SentencePiece 모델을 학습하는 과정을 거쳐야 합니다.

In [1]:
import sentencepiece as spm
import os
temp_file = os.getenv('HOME')+'/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp'

vocab_size = 8000

with open(temp_file, 'w') as f:
    for row in filtered_corpus:   # 이전 스텝에서 정제했던 corpus를 활용합니다.
        f.write(str(row) + '\n')

spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix=korean_spm --vocab_size={}'.format(temp_file, vocab_size)    
)
#위 Train에서  --model_type = 'unigram'이 디폴트 적용되어 있습니다. --model_type = 'bpe' 로 옵션을 주어 변경할 수 있습니다.

!ls -l korean_spm*

NameError: name 'filtered_corpus' is not defined

위 코드를 실행하면 정상적으로 SentencePiece 모델 학습이 완료된 후 koreanspm.model 파일과 koreanspm.vocab vocabulary 파일이 생성되었음을 확인할 수 있습니다.

그럼 이렇게 학습된 SentencePiece 모델을 어떻게 활용하는지 살펴보겠습니다.

In [2]:
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

# SentencePiece를 활용한 sentence -> encoding
tokensIDs = s.EncodeAsIds('아버지가방에들어가신다.')
print(tokensIDs)

# SentencePiece를 활용한 sentence -> encoded pieces
print(s.SampleEncodeAsPieces('아버지가방에들어가신다.',1, 0.0))

# SentencePiece를 활용한 encoding -> sentence 복원
print(s.DecodeIds(tokensIDs))

[1245, 11, 306, 7, 3598, 11, 285, 38, 3]
['▁아버지', '가', '방', '에', '들어', '가', '신', '다', '.']
아버지가방에들어가신다.


어떻습니까? SentencePiece의 토크나이징 실력이 괜찮은 것 같습니다.

## <span style="color:#124445">Step 3. Tokenizer 함수 작성</span>
---
우리는 위에서 훈련한 SentencePiece를 활용하여 위 함수와 유사한 기능을 하는 **`sp_tokenize()`** 함수를 정의할 겁니다. 하지만 SentencePiece가 동작하는 방식이 단순 토큰화와는 달라 완전히 동일하게는 정의하기 어렵습니다. 그러니 아래 조건을 만족하는 함수를 정의하도록 하겠습니다.

>*1) 매개변수로 토큰화된 문장의 **`list`** 전달하는 대신 온전한 문장의 **`list`**를 전달합니다.*<br><br>
>*2) **생성된 vocab 파일**을 읽어와 **`{ <word> : <idx> }`** 형태를 가지는 **`word_index`** 사전과 **`{ <idx> : <word>}`** 형태를 가지는 **`index_word`** 사전을 생성하고 함께 **반환**합니다.*<br><br>
>*3) 반환값인 **`tensor`**는 앞의 함수와 같이 토큰화한 후 Encoding 된 문장입니다. 바로 학습에 사용할 수 있게 Padding은 당연히 해야겠죠?*<br>

In [3]:
def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({idx:word})
        index_word.update({word:idx})

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word

In [4]:
#sp_tokenize(s, corpus) 사용예제

my_corpus = ['나는 밥을 먹었습니다.', '그러나 여전히 ㅠㅠ 배가 고픕니다...']
tensor, word_index, index_word = sp_tokenize(s, my_corpus)
print(tensor)

NameError: name 'tf' is not defined

## <span style="color:#124445">Step 4. 네이버 영화리뷰 감정분석 문제에 SentencePiece 적용해 보기</span>
---
아마 여러분들은 [네이버 영화리뷰 감정분석 태스크](https://github.com/e9t/nsmc/)를 한 번쯤은 다루어 보았을 것입니다. 한국어로 된 corpus를 다루어야 하므로 주로 KoNLPy에서 제공하는 형태소 분석기를 사용하여 텍스트를 전처리해서 RNN 모델을 분류기로 사용했을 것입니다.

만약 이 문제에서 tokenizer를 sentencepiece로 바꾸어 다시 풀어본다면 더 성능이 좋아질까요? 비교해 보는 것도 흥미로울 것입니다.

>* 네이버 영화리뷰 감정분석 코퍼스에 sentencepiece를 적용한 모델 학습하기
>*학습된 모델로 sp_tokenize() 메소드 구현하기
>*구현된 토크나이저를 적용하여 네이버 영화리뷰 감정분석 모델을 재학습하기
>*KoNLPy 형태소 분석기를 사용한 모델과 성능 비교하기
>***(보너스)** SentencePiece 모델의 model_type, vocab_size 등을 변경해 가면서 성능 개선 여부 확인하기
>*Word Vector는 활용할 필요가 없습니다. 활용이 가능하지도 않을 것입니다.
>*머지않아 SentencePiece와 BERT 등의 pretrained 모델을 함께 활용하는 태스크를 다루게 될 것입니다.

## <span style="color:#124445">Step 5. 루브릭 평가</span>
---

|평가문항|상세기준|
|-------|--------|
|1. SentencePiece를 이용하여 모델을 만들기까지의 과정이 정상적으로 진행되었는가?|코퍼스 분석, 전처리, SentencePiece 적용, 토크나이저 구현 및 동작이 빠짐없이 진행되었는가?|
|2. SentencePiece를 통해 만든 Tokenizer가 자연어처리 모델과 결합하여 동작하는가?|SentencePiece 토크나이저가 적용된 Text Classifier 모델이 정상적으로 수렴하여 80% 이상의 test accuracy가 확인되었다.|
|3. SentencePiece의 성능을 다각도로 비교·분석하였는가?|SentencePiece 토크나이저를 활용했을 때의 성능을 다른 토크나이저 혹은 SentencePiece의 다른 옵션의 경우와 비교하여 분석을 체계적으로 진행하였다.|

## <span style="color:#2C786C">Step 6. 회고</span>
---

---